This is coded on an ESP8266 motor slider
Very trivial code, no checking or async work

In [1]:
%serialconnect

Connecting to --port=/dev/ttyUSB1 --baud=115200 
Ready.


In [2]:
%sendtofile --mkdir --source /home/julian/extrepositories/micropython-lib/umqtt.robust/umqtt/robust.py lib/umqtt/
%sendtofile --mkdir --source /home/julian/extrepositories/micropython-lib/umqtt.simple/umqtt/simple.py lib/umqtt/
print(os.listdir("lib/umqtt"))


Sent 204 lines (6411 bytes) to lib/umqtt/simple.py.
['robust.py', 'simple.py']


In [3]:
%serialconnect

Connecting to --port=/dev/ttyUSB1 --baud=115200 
Ready.


In [30]:
%sendtofile main.py

import machine, time, network

wifiname = "DoESLiverpool-MQTT"
wifipassword = "decafbad00"
mqttbroker = "10.3.141.1"

from machine import Pin, I2C
import time, ustruct

time.sleep(0.1)

i2c = I2C(scl=Pin(15), sda=Pin(4), freq=450000)
#    if 0x6B in ads and 0x1E in ads:
#        res.append("SDOF GyAccMag")

        
# turn on accelerometer reg(0x20)=(ODR_XL,FS_XL,BW_SCAL_ODR,BW_XL)=110 00 0 00 should give 952Hz, but is overridden by gyros ODR 
# there are various further settings of FIFO and High and Low pass filters
i2c.writeto(0x6B, b'\x20\xC0')  
i2c.writeto(0x1E, b'\x22\x00')  

time.sleep(0.5)

def readaccelerometer():
    cs, stm, ds = 0x6B, 1, b"\x28" 
    while True:   # loop to wait for readings to be ready (at 60Hz)
        i2c.writeto(cs, b'\x27')
        st = ord(i2c.readfrom(cs, 1))  # (IG_XL,IG_G,INACT,BOOT_STATUS,TDA,GDA,XLDA) states whether a reading is ready
        if st & stm:
            break
    i2c.writeto(cs, ds)
    s = i2c.readfrom(cs, 6)
    sv = ustruct.unpack("<hhh", s)
    return sv


sta_if = network.WLAN(network.STA_IF)
sta_if.active(True)

sta_if.connect(wifiname, wifipassword)
while not sta_if.isconnected():
    time.sleep(0.1)
print("connected to wifi")

from umqtt.robust import MQTTClient
client = MQTTClient("slider", mqttbroker, 1883)
for i in range(100):
    try:
        client.connect()
        break
    except OSError as e:
        print(e)
    gopos(100+800*(i%2))
print("connected to mqtt")

import math
tdiff = 100
t0 = time.ticks_ms()
xmax = None
while True:
    #client.check_msg()
    k = readaccelerometer()
    x = math.sqrt((k[0]/16384)**2 + (k[1]/16384)**2 + (k[2]/16384)**2)
    if xmax is None or x > xmax:
        xmax = x
    t1 = time.ticks_ms()
    if t1 > t0 + tdiff and xmax is not None:
        print(xmax)
        client.publish(b"esp32black/maxaccel", str(xmax))
        xmax = None
        t0 = t1


Sent 71 lines (1882 bytes) to main.py.


In [28]:
%serialconnect

Connecting to --port=/dev/ttyUSB1 --baud=115200 
Ready.


In [29]:
import math
tdiff = 100
t0 = time.ticks_ms()
xmax = None
while True:
    #client.check_msg()
    k = readaccelerometer()
    x = math.sqrt((k[0]/16384)**2 + (k[1]/16384)**2 + (k[2]/16384)**2)
    if xmax is None or x > xmax:
        xmax = x
    t1 = time.ticks_ms()
    if t1 > t0 + tdiff and xmax is not None:
        print(xmax)
        client.publish(b"esp32black/maxaccel", str(xmax))
        xmax = None
        t0 = t1


1.055147
1.055616
1.051915
1.051115
1.051613
1.054821
1.051807
1.052601
1.06156
1.05594
1.072304
1.051569
1.051621
1.051699
1.050732
1.050541
1.052823
1.050165
1.05359
1.054039
1.052168
1.054165
1.050705
1.051476
1.058743
1.053498
1.058627
1.051066
1.148104
1.103028


*** Sending Ctrl-C



Traceback (most recent call last):
  File "<stdin>", line 9, in <module>
KeyboardInterrupt: 


In [22]:
import math
tdiff = 1000
t0 = time.ticks_ms()
xmax = None
while True:
    k = readaccelerometer()
    x = math.sqrt((k[0]/16384)**2 + (k[1]/16384)**2 + (k[2]/16384)**2)
    if xmax is None or x > xmax:
        xmax = x
    t1 = time.ticks_ms()
    if t1 > t0 + tdiff and xmax is not None:
        client.publish(b"esp32black/maxaccel", str(xmax))
        print(xmax)
        xmax = None
        t0 = t1


.1.236916
1.077793
1.280645
2.566551
1.510457
.2.716315
1.326471
1.583934
1.879438
1.076828
.1.412082
1.831047
1.240406
1.870131
2.928581
.2.526047
2.580904
1.064016
2.879952
3.027945
.1.873238
1.609989
1.088361
1.047129
1.246734
.1.044125
1.060086
1.054024
1.08132
1.057477
.1.047776
1.05077
1.053168
1.067459
1.058892
.1.046465
1.047713
1.058739
1.049314
1.046316
.1.047274
1.046779
1.051179
1.080276
1.075205
.1.069058
1.21969
1.056652
1.04788
1.054548
.1.068817
1.247896
1.054233
1.058354
1.058458
.

*** Sending Ctrl-C



Traceback (most recent call last):
  File "<stdin>", line 6, in <module>
  File "<stdin>", line 6, in readaccelerometer
KeyboardInterrupt: 


In [90]:
from umqtt.robust import MQTTClient
client = MQTTClient("slider", mqttbroker, 1883)
for i in range(100):
    try:
        client.connect()
        break
    except OSError as e:
        print(e)
    gopos(100+800*(i%2))
print("connected to mqtt")


connected to mqtt


In [95]:
client.publish(b"foo_topic", b"hello")

In [105]:

newpos = None
def sub_cb(topic, msg):
    global newpos
    print((topic, msg))
    try:
        newpos = int(msg)
    except ValueError:
        print("ignore")
        return
    newpos = min(1023, max(0, newpos))

client.set_callback(sub_cb)
client.subscribe(b"slider1/msg")


In [112]:
print(time.ticks_ms())

2638722


In [113]:
currpos = a.read()
currt0 = time.ticks_ms()
while 1:
    client.check_msg()
    if newpos is not None:
        gopos(newpos)
        newpos = None
    nextpos = a.read()
    t1 = time.ticks_ms()
    if abs(nextpos - currpos) > 10 and abs(t1 - currt0) > 200:
        print("nextpos", nextpos)
        currpos = nextpos
        currt0 = t1
        client.publish(b"slider1/pos", str(currpos).encode())


nextpos 626
nextpos 788
nextpos 823
.(b'slider1/msg', b'505')
501
nextpos 493
nextpos 406
bcn_timout,ap_probe_send_start
...............bcn_timout,ap_probe_send_start
.............bcn_timout,ap_probe_send_start
...........(b'slider1/msg', b'505')
485
nextpos 486
nextpos 542
bcn_timout,ap_probe_send_start
......(b'slider1/msg', b'540')
543
(b'slider1/msg', b'317')
340
nextpos 338
(b'slider1/msg', b'338')
328
nextpos 391
(b'slider1/msg', b'391')
391
(b'slider1/msg', b'393')
391
(b'slider1/msg', b'636')
639
nextpos 642
(b'slider1/msg', b'642')
662
nextpos 604
(b'slider1/msg', b'604')
603
.(b'slider1/msg', b'603')
604
(b'slider1/msg', b'316')
340
nextpos 339
(b'slider1/msg', b'339')
320
nextpos 387
(b'slider1/msg', b'387')
388
(b'slider1/msg', b'389')
388
(b'slider1/msg', b'589')
585
nextpos 589
nextpos 684
(b'slider1/msg', b'684')
684
(b'slider1/msg', b'690')
684
(b'slider1/msg', b'283')
276
nextpos 273
nextpos 161
(b'slider1/msg', b'273')
238
nextpos 272
(b'slider1/msg', b'161')
178
next

Traceback (most recent call last):
  File "<stdin>", line 4, in <module>
  File "/lib/umqtt/simple.py", line 204, in check_msg
  File "/lib/umqtt/robust.py", line 42, in wait_msg
  File "/lib/umqtt/robust.py", line 40, in wait_msg
  File "/lib/umqtt/simple.py", line 170, in wait_msg
KeyboardInterrupt: 


In [234]:

from machine import Pin, ADC
import time

a = ADC(0)
d5 = Pin(14, Pin.OUT)
d6 = Pin(12, Pin.OUT)
d7 = Pin(13, Pin.OUT)

def gopos(pos):
    for i in range(100):
        d = (a.read() - pos)
        if abs(d) > 40:
            di = 1 if d < 0 else 0
            d5.value(di)
            d6.value(1-di)
            d7.value(1)
        else:
            d7.value(0)
            break
        time.sleep(0.01)
    print(a.read())

import random
while True:
    pos = random.getrandbits(10)
    gopos(pos)
    time.sleep(0.5)




***Connection broken [Input/output error]
You may need to reconnect
Closing serial Serial<id=0x7fa00504b588, open=True>(port='/dev/ttyUSB3', baudrate=115200, bytesize=8, parity='N', stopbits=1, timeout=0.5, xonxoff=False, rtscts=False, dsrdtr=False)
No serial connected
  %serialconnect to connect
  %esptool to flash the device
  %lsmagic to list commands